In [1]:
from quickumls import QuickUMLS
import pandas as pd
import tqdm


In [2]:
matcher = QuickUMLS("./umls-data/out", threshold = 0.7)

In [3]:
final = pd.read_csv("final_mimic.csv")
len(final)

220429

In [4]:
#keep only the subject id, hadm id, and notes
final = final[["subject_id", "hadm_id", "text"]]

#drop duplicates because original data has multiple prescriptions for each patient in same admission or same stay
#we only want to keep distinctive notes
final = final.drop_duplicates()
len(final)

14953

In [5]:
#get all notes
all_notes = []
for i in tqdm.tqdm(range(len(final))):
    temp_dict = {}
    temp_dict["subject_id"] = note = final.iloc[i]["subject_id"]
    temp_dict["hadm_id"] = note = final.iloc[i]["hadm_id"]

    text = final.iloc[i]["text"]
    temp_dict["text"] = text

    temp_dict["matched_results"] = matcher.match(text, best_match=True, ignore_syntax=False)

    all_notes.append(temp_dict)
    
len(all_notes)

100%|██████████| 14953/14953 [7:34:51<00:00,  1.83s/it]      


14953

In [7]:
all_notes[0]

{'subject_id': np.int64(10002013),
 'hadm_id': np.int64(21763296),
 'text': " \nName:  ___                 Unit No:   ___\n \nAdmission Date:  ___              Discharge Date:   ___\n \nDate of Birth:  ___             Sex:   F\n \nService: MEDICINE\n \nAllergies: \nlisinopril\n \nAttending: ___\n \nChief Complaint:\nChest pain\n \nMajor Surgical or Invasive Procedure:\nCardiac cath (___)\n\n \nPast Medical History:\n-COPD\n-CAD s/p BMS proximal-LAD ___, DES to mid LAD ___, DES to edge \nISR of mid LAD DES and stenosis distal to stent ___, DES to OM1 \n___, s/p 3 v CABG LIMA-LAD, SVG-OM1, \n___ \n-HFpEF\n-Depression  \n-DM  \n-GERD  \n-Hypertension\n-Migraines\n-Chronic shoulder pain on narcotics\n-OSA\n-Peripheral neuropathy\n-Restless leg\n \nSocial History:\n___\nFamily History:\nPatient was ward of the state, doesn't know full details of \nfamily history. Mother with possible alcohol abuse. Father \ndeceased at ___ from Hodgkin's Disease per old records.\n \nPhysical Exam:\nADMISSIO

In [ ]:
import pickle

with open("all_notes_with_quickumls_results.pkl", "wb") as file:
    pickle.dump(all_notes, file)

## Example quickumls

In [ ]:
example_sentences = [
    "He had a CXR that demonstrated possible left base consolidation",
    "She did not have fevers or chills until the day prior to admission when she noted chills",
    "At least moderate pulmonary hypertension",
    "He was then brsought to the [**Hospital118**] ED for further management",
    "Metastatic osteogenic sarcoma"
]
for text in example_sentences:
    print(text)
    print("\n")
    print(matcher.match(text, best_match=True, ignore_syntax=False))
    print("\n\n")


He had a CXR that demonstrated possible left base consolidation


[[{'start': 31, 'end': 39, 'ngram': 'possible', 'term': 'possible', 'cui': 'C0332149', 'similarity': 1.0, 'semtypes': {'T033'}, 'preferred': 1}], [{'start': 9, 'end': 12, 'ngram': 'CXR', 'term': 'cxr', 'cui': 'C0039985', 'similarity': 1.0, 'semtypes': {'T060'}, 'preferred': 1}], [{'start': 50, 'end': 63, 'ngram': 'consolidation', 'term': 'Consolidation', 'cui': 'C0521530', 'similarity': 0.8333333333333334, 'semtypes': {'T047'}, 'preferred': 1}], [{'start': 18, 'end': 30, 'ngram': 'demonstrated', 'term': 'Not demonstrated', 'cui': 'C4697740', 'similarity': 0.7142857142857143, 'semtypes': {'T033'}, 'preferred': 1}]]



She did not have fevers or chills until the day prior to admission when she noted chills


[[{'start': 57, 'end': 66, 'ngram': 'admission', 'term': 'admission', 'cui': 'C0184666', 'similarity': 1.0, 'semtypes': {'T058'}, 'preferred': 1}, {'start': 57, 'end': 66, 'ngram': 'admission', 'term': 'admissions', 'c